# Modifying data in-place
---
in-place: 추가적인 메모리를 사용하지 않고 기존의 데이터를 수정   
-> 적절히 사용하면 컴퓨팅 계산 때 필요한 메모리 용량을 줄일 수 있다. 하지만 반대로는 예상하지 못한 결과가 나올 수도 있다.   
**-> 언제 사용해야 하고 언제 사용하면 안되는 지를 배우자!**

In [1]:
# 예시 데이터
import mne

sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = sample_data_folder / "MEG" / "sample" / "sample_audvis_raw.fif"
# the preload flag loads the data into memory now
raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True)
raw.crop(tmax=10.0)  # raw.crop() always happens in-place

Opening raw data file C:\Users\supli\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...


<Raw | sample_audvis_raw.fif, 376 x 6007 (10.0 s), ~20.5 MB, data loaded>

#### Signal processing

대부분의 MNE-python 오브젝트는 필터링, 힐버트 변환 등의 method가 내장되어 있다.   
이 method들은 기본적으로 in-place modify를 지원한다.   
그러므로 원본 데이터와 수정된 데이터를 비교하기 전에 원본 데이터의 사본을 만들어 놓아야만 한다.

In [2]:
original_raw = raw.copy()
raw.apply_hilbert()
print(
    f"original data type was {original_raw.get_data().dtype}, after "
    f"apply_hilbert the data type changed to {raw.get_data().dtype}."
)

original data type was float64, after apply_hilbert the data type changed to complex128.


원본 데이터가 힐버트 변환 후에 ```float64``` 에서 ```complex128``` 로 변한 것을 알 수 있다. (modify in-place)

#### Channel picking 

modify in-place가 적용되는 또 다른 일련의 methods는 channel-picking methods 이다.   
(channel-picking: 데이터 분석 시 특정한 채널을 선택하여 작업 수행)



In [3]:
print(f'original data had {original_raw.info["nchan"]} channels.')
original_raw.pick("eeg")  # selects only the EEG channels
print(f'after picking, it has {original_raw.info["nchan"]} channels.')

original data had 376 channels.
Removing projector <Projection | PCA-v1, active : False, n_channels : 102>
Removing projector <Projection | PCA-v2, active : False, n_channels : 102>
Removing projector <Projection | PCA-v3, active : False, n_channels : 102>
after picking, it has 60 channels.


EEG picking 후 원본 데이터가 376 channel 에서 60 channel로 줄은 것을 알 수 있다.

#### The copy parameter 

그러므로 modify in-place에 대비하기 위해 ```copy``` method를 사용해야 한다.   
하지만 대부분의 MNE-python에는 copy parameter가 내장되어 있다. 
> ```copy=True``` : 사본 데이터를 반환   
> ```copy=false```: modify in-place




In [4]:
rereferenced_raw, ref_data = mne.set_eeg_reference(original_raw, ["EEG 003"], copy=True)
fig_orig = original_raw.plot()
fig_reref = rereferenced_raw.plot()

EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Using qt as 2D backend.


<table>
  <tr>
    <td><img src="./img/inductory_tutorials/copy_parmeter_ex1.png" width=400 /></td><td><img src="./img/inductory_tutorials/copy_parmeter_ex2.png" width=400 /></td>
  <tr>
</table>

```set_eeg_reference()```를 실행했을 때 ```copy``` parameter에 ```True``` 값을 넣었기 때문에 mehtod 실행 후에도 원본 데이터가 남아있는 것을 확인할 수 있다.   
-> 비교 분석 시에는 이런 식으로 해야 함.

#### Summary

object의 method인 경우에는 in-place modify가 기본적으로 실행될 것을 숙지해야 하고,   
obeject가 parmeter 값에 들어가는 function의 경우에도 ```copy``` parameter를 잘 확인해야 한다.    
그러므로 ```copy()``` method를 잘 활용하여 이에 대비하는 것이 중요하다.